# Unified Experiment Code
> Resistance is futile.

In [ ]:
# default_exp unified_experiment

In [ ]:
# hide
import blackhc.project.script

Import modules and functions were are going to use.

In [ ]:
# exports

import dataclasses
import traceback
from dataclasses import dataclass
from typing import Type, Union, Optional

import torch
import torch.utils.data
from blackhc.project import is_run_from_ipython
from blackhc.project.experiment import embedded_experiments

import batchbald_redux.acquisition_functions as acquisition_functions
from batchbald_redux.acquisition_functions import (
    CandidateBatchComputer,
    EvalCandidateBatchComputer,
)
from batchbald_redux.black_box_model_training import evaluate
from batchbald_redux.dataset_challenges import (
    get_base_dataset_index,
    get_target,
)
from batchbald_redux.di import DependencyInjection
from batchbald_redux.experiment_data import ExperimentData, ExperimentDataConfig, OoDDatasetConfig
from batchbald_redux.models import MnistModelTrainer
from batchbald_redux.resnet_models import Cifar10ModelTrainer
from batchbald_redux.train_eval_model import (
    TrainEvalModel,
    TrainSelfDistillationEvalModel,
)
from batchbald_redux.trained_model import ModelTrainer

In [ ]:
# exports


@dataclass
class UnifiedExperiment:
    seed: int

    id_dataset_name: str
    ood_dataset_name: Optional[str]
    ood_exposure: bool
    initial_training_set_size: int = 20
    validation_set_size: int = 1024
    evaluation_set_size: int = 1024
    id_repetitions: float = 1
    ood_repetitions: float = 1
    add_dataset_noise: bool = False
    validation_split_random_state: int = 0

    acquisition_size: int = 5
    max_training_set: int = 200

    max_training_epochs: int = 300

    num_pool_samples: int = 100
    num_validation_samples: int = 20
    num_training_samples: int = 1

    device: str = "cuda"
    acquisition_function: Union[
        Type[CandidateBatchComputer], Type[EvalCandidateBatchComputer]
    ] = acquisition_functions.BALD
    train_eval_model: Type[TrainEvalModel] = TrainSelfDistillationEvalModel
    model_trainer_factory: Type[ModelTrainer] = Cifar10ModelTrainer

    temperature: float = 0.0

    def load_experiment_data(self) -> ExperimentData:
        di = DependencyInjection(vars(self), [])
        odc: OoDDatasetConfig = di.create_dataclass_type(OoDDatasetConfig) if self.ood_dataset_name is not None else None
        edc: ExperimentDataConfig = di.create_dataclass_type(ExperimentDataConfig, ood_dataset_config=odc)
        return edc.load()

    # Simple Dependency Injection
    def create_acquisition_function(self):
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.acquisition_function)

    def create_train_eval_model(self, runtime_config) -> TrainEvalModel:
        config = {**vars(self), **runtime_config}
        di = DependencyInjection(config, [])
        return di.create_dataclass_type(self.train_eval_model)

    def create_model_trainer(self) -> ModelTrainer:
        di = DependencyInjection(vars(self))
        return di.create_dataclass_type(self.model_trainer_factory)

    def run(self, store):
        torch.manual_seed(self.seed)

        # Active Learning setup
        data = self.load_experiment_data()
        store["dataset_info"] = dict(training=repr(data.active_learning.base_dataset), test=repr(data.test_dataset))
        store["initial_training_set_indices"] = data.initial_training_set_indices
        store["evaluation_set_indices"] = data.evaluation_set_indices

        model_trainer = self.create_model_trainer()

        train_loader = model_trainer.get_train_dataloader(data.active_learning.training_dataset)
        pool_loader = model_trainer.get_evaluation_dataloader(data.active_learning.pool_dataset)
        validation_loader = model_trainer.get_evaluation_dataloader(data.validation_dataset)
        test_loader = model_trainer.get_evaluation_dataloader(data.test_dataset)

        store["active_learning_steps"] = []
        active_learning_steps = store["active_learning_steps"]

        acquisition_function = self.create_acquisition_function()

        num_iterations = 0
        max_iterations = int(1.5 * (self.max_training_set - self.initial_training_set_size) / self.acquisition_size)

        # Active Training Loop
        while True:
            training_set_size = len(data.active_learning.training_dataset)
            print(f"Training set size {training_set_size}:")

            # iteration_log = dict(training={}, pool_training={}, evaluation_metrics=None, acquisition=None)
            active_learning_steps.append({})
            iteration_log = active_learning_steps[-1]

            iteration_log["training"] = {}

            # TODO: this is a hack! :(
            if self.ood_dataset_name is None:
                loss = validation_loss = torch.nn.NLLLoss()
            elif self.ood_exposure:
                loss = torch.nn.KLDivLoss(log_target=False, reduction="batchmean")
                validation_loss = torch.nn.NLLLoss()
            else:
                loss = validation_loss = torch.nn.NLLLoss()

            trained_model = model_trainer.get_trained(train_loader=train_loader, train_augmentations=data.train_augmentations,
                                                      validation_loader=validation_loader,
                                                      log=iteration_log["training"], loss=loss,
                                                      validation_loss=validation_loss)

            evaluation_metrics = evaluate(model=trained_model, num_samples=self.num_validation_samples,
                                          loader=test_loader, device=self.device, storage_device="cpu")
            iteration_log["evaluation_metrics"] = evaluation_metrics
            print(f"Perf after training {evaluation_metrics}")

            if training_set_size >= self.max_training_set or num_iterations >= max_iterations:
                print("Done.")
                break

            if isinstance(acquisition_function, CandidateBatchComputer):
                candidate_batch = acquisition_function.compute_candidate_batch(trained_model, pool_loader, self.device)
            elif isinstance(acquisition_function, EvalCandidateBatchComputer):
                if self.evaluation_set_size:
                    eval_dataset = data.evaluation_dataset
                else:
                    eval_dataset = data.active_learning.pool_dataset

                train_eval_model = self.create_train_eval_model(
                    dict(
                        model_trainer=model_trainer,
                        training_dataset=data.active_learning.training_dataset,
                        train_augmentations=data.train_augmentations,
                        eval_dataset=eval_dataset,
                        validation_loader=validation_loader,
                        trained_model=trained_model,
                        dataset_device=data.device
                    )
                )

                iteration_log["eval_training"] = {}
                trained_eval_model = train_eval_model(device=self.device, training_log=iteration_log["eval_training"])

                candidate_batch = acquisition_function.compute_candidate_batch(
                    trained_model, trained_eval_model, pool_loader, device=self.device
                )
            else:
                raise ValueError(f"Unknown acquisition function {acquisition_function}!")

            candidate_global_dataset_indices = []
            candidate_labels = []
            for index in candidate_batch.indices:
                base_di = get_base_dataset_index(data.active_learning.pool_dataset, index)
                dataset_type = "ood" if base_di.dataset == data.ood_dataset else "id"
                candidate_global_dataset_indices.append((dataset_type, base_di.index))
                label = get_target(data.active_learning.pool_dataset, index).cpu().numpy()
                candidate_labels.append(label)

            iteration_log["acquisition"] = dict(
                indices=candidate_global_dataset_indices, labels=candidate_labels, scores=candidate_batch.scores
            )

            print(candidate_batch)
            print(candidate_global_dataset_indices)

            if self.ood_dataset_name is None:
                data.active_learning.acquire(candidate_batch.indices)
            elif self.ood_exposure:
                data.active_learning.acquire(candidate_batch.indices)
            else:
                data.active_learning.acquire(
                    [index for index, label in zip(candidate_batch.indices, candidate_labels) if label != -1]
                )

            ls = ", ".join(f"{label} ({score:.4})" for label, score in zip(candidate_labels, candidate_batch.scores))
            print(f"Acquiring (label, score)s: {ls}")

            num_iterations += 1

## CIFAR-10 vs SVHN Coverage

In [ ]:
# experiment
# OOD experiment (ood_exposure=True)

experiment = UnifiedExperiment(
    ood_exposure=True,
    id_dataset_name="CIFAR-10",
    ood_dataset_name="SVHN",
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: OoDDatasetConfig(
	ood_dataset_name=SVHN,
	ood_repetitions=1,
	ood_exposure=True
)
Creating: ExperimentDataConfig(
	id_dataset_name=CIFAR-10,
	id_repetitions=1,
	initial_training_set_size=0,
	validation_set_size=1024,
	validation_split_random_state=0,
	evaluation_set_size=100,
	add_dataset_noise=False,
	device=cuda,
	ood_dataset_config=OoDDatasetConfig(ood_dataset_name='SVHN', ood_repetitions=1, ood_exposure=True)
)
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/test_32x32.mat
Creating: Cifar10ModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Training set size 0:
Cosine Annealing


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.0991, 'crossentropy': tensor(2.3396)}
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2,
	training_dataset=<torch.utils.data.dataset.Subset object at 0x7f4e0402d250>,
	eval_dataset=Evaluation Set (100 samples),
	validation_loader=<torch.utils.data.dataloader.DataLoader object at 0x7f4e0402dcd0>,
	trained_model=TrainedBayesianModel(model=BayesianResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2)

get_predictions_labels:   0%|          | 0/200 [00:00<?, ?it/s]

Cosine Annealing


...

[1/39]   3%|2          [00:00<?]

Epoch 1 metrics: {'accuracy': 0.1630859375, 'crossentropy': 3.1745386123657227}


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.1488, 'crossentropy': tensor(3.0583)}
Done.


{'dataset_info': {'training': "'CIFAR-10 (Train, seed=0, 48976 samples)' | one_hot_targets{'num_classes': 10} + 'SVHN (Train, seed=0, 73257 samples)' | uniform_targets{'num_classes': 10}",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [],
 'evaluation_set_indices': [12980,
  44617,
  3812,
  42704,
  6729,
  38942,
  48125,
  16968,
  5652,
  4045,
  6984,
  21168,
  37164,
  33354,
  47307,
  17878,
  26665,
  40819,
  14805,
  201,
  33976,
  35571,
  15578,
  36667,
  5551,
  23088,
  32496,
  5705,
  23255,
  25559,
  33058,
  43729,
  47518,
  36303,
  18452,
  34447,
  24821,
  36157,
  48089,
  25120,
  26944,
  24745,
  11001,
  6995,
  10899,
  36881,
  7002,
  19049,
  13388,
  40737,
  66,
  14046,
  45656,
  5604,
  9134,
  35979,
  19757,
  43627,
  35248,
  23566,
  46542,
  39478,
  25443,
  45862,
  30730,
  9611,
  43077,
  23902,
  9541,
  38859,
  6000,
  5915,
  21547,
  47739,
  13909,
  624,
  25621,
  30173,
  37807,
  7053,
  39360,
  20774,
 

In [ ]:
# experiment
# OOD experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    ood_exposure=False,
    id_dataset_name="CIFAR-10",
    ood_dataset_name="SVHN",
    seed=1,
    max_training_epochs=1,
    max_training_set=20 + 10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: OoDDatasetConfig(
	ood_dataset_name=SVHN,
	ood_repetitions=1,
	ood_exposure=False
)
Creating: ExperimentDataConfig(
	id_dataset_name=CIFAR-10,
	id_repetitions=1,
	initial_training_set_size=0,
	validation_set_size=1024,
	validation_split_random_state=0,
	evaluation_set_size=100,
	add_dataset_noise=False,
	device=cuda,
	ood_dataset_config=OoDDatasetConfig(ood_dataset_name='SVHN', ood_repetitions=1, ood_exposure=False)
)
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: data/SVHN/train_32x32.mat
Using downloaded and verified file: data/SVHN/test_32x32.mat
Creating: Cifar10ModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Training set size 0:
Cosine Annealing


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.0991, 'crossentropy': tensor(2.3396)}
Creating: TrainSelfDistillationEvalModel(
	num_pool_samples=2,
	training_dataset=<torch.utils.data.dataset.Subset object at 0x7f4e4fe8aca0>,
	eval_dataset=Evaluation Set (100 samples),
	validation_loader=<torch.utils.data.dataloader.DataLoader object at 0x7f4e4fe8a4f0>,
	trained_model=TrainedBayesianModel(model=BayesianResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2)

get_predictions_labels:   0%|          | 0/200 [00:00<?, ?it/s]

Cosine Annealing


...

[1/39]   3%|2          [00:00<?]

Epoch 1 metrics: {'accuracy': 0.15625, 'crossentropy': 4.910492897033691}


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.1531, 'crossentropy': tensor(4.3525)}
Done.


{'dataset_info': {'training': "('CIFAR-10 (Train, seed=0, 48976 samples)') + 'SVHN (Train, seed=0, 73257 samples)' | constant_target{'target': tensor(-1), 'num_classes': 10}",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [],
 'evaluation_set_indices': [12980,
  44617,
  3812,
  42704,
  6729,
  38942,
  48125,
  16968,
  5652,
  4045,
  6984,
  21168,
  37164,
  33354,
  47307,
  17878,
  26665,
  40819,
  14805,
  201,
  33976,
  35571,
  15578,
  36667,
  5551,
  23088,
  32496,
  5705,
  23255,
  25559,
  33058,
  43729,
  47518,
  36303,
  18452,
  34447,
  24821,
  36157,
  48089,
  25120,
  26944,
  24745,
  11001,
  6995,
  10899,
  36881,
  7002,
  19049,
  13388,
  40737,
  66,
  14046,
  45656,
  5604,
  9134,
  35979,
  19757,
  43627,
  35248,
  23566,
  46542,
  39478,
  25443,
  45862,
  30730,
  9611,
  43077,
  23902,
  9541,
  38859,
  6000,
  5915,
  21547,
  47739,
  13909,
  624,
  25621,
  30173,
  37807,
  7053,
  39360,
  20774,
  23760,
  265

## MNIST vs FashionMNIST Coverage

In [ ]:
# experiment
# MNIST OOD experiment (ood_exposure=True)

experiment = UnifiedExperiment(
    ood_exposure=True,
    id_dataset_name="MNIST",
    ood_dataset_name="FashionMNIST",
    seed=1,
    max_training_epochs=1,
    max_training_set=20+10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: OoDDatasetConfig(
	ood_dataset_name=FashionMNIST,
	ood_repetitions=1,
	ood_exposure=True
)
Creating: ExperimentDataConfig(
	id_dataset_name=MNIST,
	id_repetitions=1,
	initial_training_set_size=20,
	validation_set_size=1024,
	validation_split_random_state=0,
	evaluation_set_size=100,
	add_dataset_noise=False,
	device=cuda,
	ood_dataset_config=OoDDatasetConfig(ood_dataset_name='FashionMNIST', ood_repetitions=1, ood_exposure=True)
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=1
)
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Training set size 20:


100%|##########| 1/1 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.49609375, 'crossentropy': 1.8818157464265823}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.49609375)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

...

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.3564453125, 'crossentropy': 2.0059348344802856}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.3564453125)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.3921, 'crossentropy': tensor(1.7375)}
Done.


{'dataset_info': {'training': "'MNIST (Train, seed=0, 58976 samples)' | one_hot_targets{'num_classes': 10} + 'FashionMNIST (Train, seed=0, 60000 samples)' | uniform_targets{'num_classes': 10}",
  'test': "'MNIST (Test)'"},
 'initial_training_set_indices': [53434,
  8533,
  14640,
  39579,
  30392,
  58125,
  37915,
  3091,
  57520,
  43803,
  44119,
  52296,
  58226,
  40334,
  46037,
  22015,
  22304,
  43812,
  12640,
  53689],
 'evaluation_set_indices': [29974,
  55573,
  35472,
  44048,
  48031,
  5616,
  10110,
  47420,
  56990,
  34198,
  3792,
  5715,
  15969,
  32775,
  19757,
  34588,
  28991,
  47417,
  26501,
  12108,
  5573,
  48032,
  40646,
  43252,
  2404,
  36797,
  29079,
  40018,
  37047,
  41512,
  45567,
  801,
  10664,
  52801,
  42890,
  32972,
  45974,
  20801,
  23496,
  5803,
  10508,
  46870,
  49549,
  306,
  38725,
  13074,
  19689,
  27135,
  16068,
  18137,
  2728,
  43321,
  29950,
  380,
  27254,
  50466,
  31965,
  24052,
  44454,
  20076,
  21423,
  58

## MNIST only

In [ ]:
# experiment
# MNIST experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    ood_exposure=False,
    id_dataset_name="MNIST",
    ood_dataset_name=None,
    seed=1,
    max_training_epochs=5,
    max_training_set=20+10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    model_trainer_factory=MnistModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: ExperimentDataConfig(
	id_dataset_name=MNIST,
	id_repetitions=1,
	initial_training_set_size=20,
	validation_set_size=1024,
	validation_split_random_state=0,
	evaluation_set_size=100,
	add_dataset_noise=False,
	device=cuda,
	ood_dataset_config=None
)
Creating: MnistModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Training set size 20:


 20%|##        | 1/5 [00:00<?, ?it/s]

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.494140625, 'crossentropy': 1.8813515156507492}


[1/16]   6%|6          [00:00<?]

...

[1/16]   6%|6          [00:00<?]

Epoch metrics: {'accuracy': 0.6123046875, 'crossentropy': 2.385873556137085}
RestoringEarlyStopping: Restoring best parameters. (Score: 0.6123046875)
RestoringEarlyStopping: Restoring optimizer.


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.6413, 'crossentropy': tensor(1.1311)}
Done.


{'dataset_info': {'training': "'MNIST (Train, seed=0, 58976 samples)'",
  'test': "'MNIST (Test)'"},
 'initial_training_set_indices': [53434,
  8533,
  14640,
  39579,
  30392,
  58125,
  37915,
  3091,
  57520,
  43803,
  44119,
  52296,
  58226,
  40334,
  46037,
  22015,
  22304,
  43812,
  12640,
  53689],
 'evaluation_set_indices': [29974,
  55573,
  35472,
  44048,
  48031,
  5616,
  10110,
  47420,
  56990,
  34198,
  3792,
  5715,
  15969,
  32775,
  19757,
  34588,
  28991,
  47417,
  26501,
  12108,
  5573,
  48032,
  40646,
  43252,
  2404,
  36797,
  29079,
  40018,
  37047,
  41512,
  45567,
  801,
  10664,
  52801,
  42890,
  32972,
  45974,
  20801,
  23496,
  5803,
  10508,
  46870,
  49549,
  306,
  38725,
  13074,
  19689,
  27135,
  16068,
  18137,
  2728,
  43321,
  29950,
  380,
  27254,
  50466,
  31965,
  24052,
  44454,
  20076,
  21423,
  58741,
  27145,
  38430,
  37354,
  49986,
  4321,
  12610,
  34482,
  35794,
  396,
  50036,
  46861,
  57811,
  53831,
  4

## CIFAR-10 only

In [ ]:
# experiment
# CIFAR-10 experiment (ood_exposure=False)

experiment = UnifiedExperiment(
    ood_exposure=False,
    id_dataset_name="CIFAR-10",
    ood_dataset_name=None,
    seed=1,
    max_training_epochs=5,
    max_training_set=20+10,
    acquisition_function=acquisition_functions.EvalBALD,
    evaluation_set_size=100,
    acquisition_size=10,
    model_trainer_factory=Cifar10ModelTrainer,
    num_pool_samples=2,
    device="cuda",
)

results = {}
experiment.run(results)
results

Creating: ExperimentDataConfig(
	id_dataset_name=CIFAR-10,
	id_repetitions=1,
	initial_training_set_size=20,
	validation_set_size=1024,
	validation_split_random_state=0,
	evaluation_set_size=100,
	add_dataset_noise=False,
	device=cuda,
	ood_dataset_config=None
)
Files already downloaded and verified
Files already downloaded and verified
Creating: Cifar10ModelTrainer(
	device=cuda,
	num_training_samples=1,
	num_validation_samples=20,
	max_training_epochs=5
)
Creating: EvalBALD(
	acquisition_size=10,
	num_pool_samples=2
)
Training set size 20:
Cosine Annealing


 20%|##        | 1/5 [00:00<?, ?it/s]

[1/39]   3%|2          [00:00<?]

Epoch 1 metrics: {'accuracy': 0.1552734375, 'crossentropy': 5.018630504608154}


[1/39]   3%|2          [00:00<?]

...

[1/39]   3%|2          [00:00<?]

Epoch 5 metrics: {'accuracy': 0.2080078125, 'crossentropy': 8.315721988677979}


get_predictions_labels:   0%|          | 0/200000 [00:00<?, ?it/s]

Perf after training {'accuracy': 0.2031, 'crossentropy': tensor(6.3103)}
Done.


{'dataset_info': {'training': "'CIFAR-10 (Train, seed=0, 48976 samples)'",
  'test': "'CIFAR-10 (Test)'"},
 'initial_training_set_indices': [12980,
  44617,
  6984,
  21168,
  33976,
  35571,
  33058,
  43729,
  26944,
  24745,
  66,
  14046,
  46542,
  39478,
  6000,
  5915,
  39360,
  20774,
  27084,
  44464],
 'evaluation_set_indices': [3812,
  42704,
  6729,
  38942,
  48125,
  16968,
  5652,
  4045,
  10740,
  19606,
  37164,
  33354,
  47307,
  17878,
  26665,
  40819,
  14805,
  201,
  47956,
  44739,
  15578,
  36667,
  5551,
  23088,
  32496,
  5705,
  23255,
  25559,
  11975,
  44032,
  47518,
  36303,
  18452,
  34447,
  24821,
  36157,
  48089,
  25120,
  44689,
  6509,
  11001,
  6995,
  10899,
  36881,
  7002,
  19049,
  13388,
  40737,
  9210,
  22684,
  45656,
  5604,
  9134,
  35979,
  19757,
  43627,
  35248,
  23566,
  727,
  34909,
  25443,
  45862,
  30730,
  9611,
  43077,
  23902,
  9541,
  38859,
  13973,
  27923,
  21547,
  47739,
  13909,
  624,
  25621,
  301

In [ ]:
# exports

configs = [
    UnifiedExperiment(
        seed=seed + 1234,
        ood_exposure=ood_exposure,
        acquisition_function=acquisition_function,
        acquisition_size=5,
        num_pool_samples=num_pool_samples,
        evaluation_set_size=evaluation_set_size,
        id_dataset_name=id_dataset_name,
        ood_dataset_name=ood_dataset_name,
    )
    for seed in range(3)
    for acquisition_function in [acquisition_functions.BatchEvalBALD, acquisition_functions.BatchBALD]
    for evaluation_set_size in [1024]
    for num_pool_samples in [100]
    for ood_exposure in [True, False]
    for id_dataset_name, ood_dataset_name in [("CIFAR-10", "SVHN"), ("SVHN", "CIFAR-10")]
]

if not is_run_from_ipython() and __name__ == "__main__":
    for job_id, store in embedded_experiments(__file__, len(configs)):
        config = configs[job_id]
        config.seed += job_id
        print(config)
        store["config"] = dataclasses.asdict(config)
        store["log"] = {}

        try:
            config.run(store=store)
        except Exception:
            store["exception"] = traceback.format_exc()
            raise

In [ ]:
len(configs)

24

In [ ]:
# slow
import prettyprinter

prettyprinter.install_extras(include={"dataclasses"})

prettyprinter.pprint(configs)

[
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=True,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='CIFAR-10',
        ood_dataset_name='SVHN',
        num_pool_samples=100,
        # class
        acquisition_function=batchbald_redux.acquisition_functions.BatchEvalBALD
    ),
    OodExperiment(
        seed=1234,
        uniform_ood=False,
        id_dataset_name='SVHN',
        ood_dataset_name='CIFAR-10',
        num_pool_samples=100,
        # class
        acquisit